In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
%matplotlib inline
sns.set_style('white')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [2]:
#Taking the NJ Crime FBI data
NJ2013 = pd.read_csv("NJ2013.csv")

In [3]:
NJ2013.head()

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NEW JERSEY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"by City, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson,NaN
4,Aberdeen Township,"18,150",19,0,NaN,0,13,6,237,42,185,10,0,NaN


In [4]:
#Looks like the is no proper defined headder so drop first 4 rows
NJ2013.drop(NJ2013.index[:4], inplace=True)

In [5]:
NJ2013.head()

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
4,Aberdeen Township,"18,150",19,0,NaN,0,13,6,237,42,185,10,0,NaN
5,Absecon,"8,380",21,0,NaN,2,4,15,266,91,169,6,0,NaN
6,Allendale,"6,712",0,0,NaN,0,0,0,37,6,29,2,0,NaN
7,Allenhurst,493,0,0,NaN,0,0,0,39,20,18,1,0,NaN
8,Allentown,"1,812",3,0,NaN,0,0,3,18,12,6,0,0,NaN


In [6]:
#Give meaningful names for columns
NJ2013.columns = ['City','Population','Vcrime','Murder','Raperev','Rape','Robbery','Assault','Pcrime',
                 'Burglary','Larceny','Theft','Arson','Temp']

In [7]:
NJ2013.head()

,City,Population,Vcrime,Murder,Raperev,Rape,Robbery,Assault,Pcrime,Burglary,Larceny,Theft,Arson,Temp
4,Aberdeen Township,"18,150",19,0,NaN,0,13,6,237,42,185,10,0,NaN
5,Absecon,"8,380",21,0,NaN,2,4,15,266,91,169,6,0,NaN
6,Allendale,"6,712",0,0,NaN,0,0,0,37,6,29,2,0,NaN
7,Allenhurst,493,0,0,NaN,0,0,0,39,20,18,1,0,NaN
8,Allentown,"1,812",3,0,NaN,0,0,3,18,12,6,0,0,NaN


In [8]:
#Look for the datatypes
NJ2013.dtypes

City           object
Population     object
Vcrime         object
Murder         object
Raperev        object
Rape           object
Robbery        object
Assault        object
Pcrime         object
Burglary       object
Larceny        object
Theft          object
Arson          object
Temp          float64
dtype: object

In [9]:
#Before changing the datatype to float change the format for numbers by removing commas
NJ2013.iloc[:,1:13] = NJ2013.iloc[:,1:13].replace(
    to_replace=[','],
    value='',
    regex=True
)

In [10]:
NJ2013

,City,Population,Vcrime,Murder,Raperev,Rape,Robbery,Assault,Pcrime,Burglary,Larceny,Theft,Arson,Temp
4,Aberdeen Township,18150,19,0,NaN,0,13,6,237,42,185,10,0,NaN
5,Absecon,8380,21,0,NaN,2,4,15,266,91,169,6,0,NaN
6,Allendale,6712,0,0,NaN,0,0,0,37,6,29,2,0,NaN
7,Allenhurst,493,0,0,NaN,0,0,0,39,20,18,1,0,NaN
8,Allentown,1812,3,0,NaN,0,0,3,18,12,6,0,0,NaN
9,Alpha,2314,1,0,NaN,0,0,1,31,15,15,1,1,NaN
10,Alpine,1958,0,0,NaN,0,0,0,10,6,2,2,0,NaN
11,Andover Township,6273,1,0,NaN,0,0,1,33,10,23,0,0,NaN
12,Asbury Park,15779,264,6,NaN,9,126,123,842,196,616,30,3,NaN
13,Atlantic City,39482,685,3,NaN,9,367,306,2475,393,2011,71,2,NaN


In [11]:
#Now change datattypes to float
NJ2013.iloc[:,1:13] = NJ2013.iloc[:,1:13].apply(lambda x:x.astype(float))

In [12]:
#All good?
NJ2013.dtypes

City           object
Population    float64
Vcrime        float64
Murder        float64
Raperev       float64
Rape          float64
Robbery       float64
Assault       float64
Pcrime        float64
Burglary      float64
Larceny       float64
Theft         float64
Arson         float64
Temp          float64
dtype: object

In [13]:
#Last two rows need to go
NJ2013.drop(NJ2013.tail(2).index,inplace=True)

In [14]:
#All good?
NJ2013

,City,Population,Vcrime,Murder,Raperev,Rape,Robbery,Assault,Pcrime,Burglary,Larceny,Theft,Arson,Temp
4,Aberdeen Township,18150.0,19.0,0.0,NaN,0.0,13.0,6.0,237.0,42.0,185.0,10.0,0.0,NaN
5,Absecon,8380.0,21.0,0.0,NaN,2.0,4.0,15.0,266.0,91.0,169.0,6.0,0.0,NaN
6,Allendale,6712.0,0.0,0.0,NaN,0.0,0.0,0.0,37.0,6.0,29.0,2.0,0.0,NaN
7,Allenhurst,493.0,0.0,0.0,NaN,0.0,0.0,0.0,39.0,20.0,18.0,1.0,0.0,NaN
8,Allentown,1812.0,3.0,0.0,NaN,0.0,0.0,3.0,18.0,12.0,6.0,0.0,0.0,NaN
9,Alpha,2314.0,1.0,0.0,NaN,0.0,0.0,1.0,31.0,15.0,15.0,1.0,1.0,NaN
10,Alpine,1958.0,0.0,0.0,NaN,0.0,0.0,0.0,10.0,6.0,2.0,2.0,0.0,NaN
11,Andover Township,6273.0,1.0,0.0,NaN,0.0,0.0,1.0,33.0,10.0,23.0,0.0,0.0,NaN
12,Asbury Park,15779.0,264.0,6.0,NaN,9.0,126.0,123.0,842.0,196.0,616.0,30.0,3.0,NaN
13,Atlantic City,39482.0,685.0,3.0,NaN,9.0,367.0,306.0,2475.0,393.0,2011.0,71.0,2.0,NaN


In [15]:
#Drop the columns Temp and Raperev has many Nan and temp is not even real column
NJ=NJ2013.drop(['Raperev','Temp'],axis=1)

In [16]:
#Check for any null values
NJ.describe()

,Population,Vcrime,Murder,Rape,Robbery,Assault,Pcrime,Burglary,Larceny,Theft,Arson
count,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000
mean,17556.910230,47.407098,0.703549,1.607516,23.077244,22.018789,323.008351,69.338205,226.638831,27.031315,1.083507
std,25179.212668,213.254026,5.619360,4.290744,131.534269,79.018890,643.365923,155.785117,380.687208,152.196186,3.137924
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4807.500000,2.000000,0.000000,0.000000,0.000000,1.000000,49.000000,10.000000,36.000000,1.000000,0.000000
50%,9613.000000,7.000000,0.000000,0.000000,2.000000,4.000000,134.000000,26.000000,100.000000,4.000000,0.000000
75%,21432.500000,21.000000,0.000000,1.000000,9.000000,12.000000,335.000000,60.000000,244.500000,13.500000,1.000000
max,278246.000000,3516.000000,112.000000,45.000000,2433.000000,926.000000,8965.000000,2074.000000,3997.000000,2894.000000,34.000000


In [17]:
#Max is 2too much for population, lets take 75%
NJ = NJ[NJ.Population < (np.percentile(NJ.Population, 75))]

In [18]:
#Take the median for property crime which will be target
print(np.median(NJ.Pcrime))

85.0


In [19]:
#Convert Property crime to categorical feature
NJ['Pcrime_cat'] = np.where(NJ.Pcrime >= 85,1,0)

In [20]:
y = NJ.Pcrime_cat.values.reshape(-1, 1)
X = NJ.loc[:, ~(NJ.columns).isin(['City','Pcrime','Pcrime_cat'])]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Lasso Regression

In [22]:
# Declare and fit the model
# Set regularization parameter
# turn down tolerance for short training time
clf_l1_LR = LogisticRegression(C=0.05, penalty='l1')

clf_l1_LR.fit(X_train, y_train)

print("Coefficient: ", clf_l1_LR.coef_)
print("Intercept:",clf_l1_LR.intercept_)
print("Train score with L1 penalty: %.4f" % clf_l1_LR.score(X_train, y_train))
print("Test score with L1 penalty: %.4f" % clf_l1_LR.score(X_test, y_test))
print("Cross Validation Scores: ",cross_val_score(clf_l1_LR, X_train, y_train, cv=10))
print("Cross Validation mean: ",cross_val_score(clf_l1_LR, X_train, y_train, cv=10).mean())

Coefficient:  [[-0.00010437  0.          0.          0.          0.          0.
   0.01775952  0.03998706  0.02910591  0.        ]]
Intercept: [-1.83079831]
Train score with L1 penalty: 0.9402
Test score with L1 penalty: 0.9259
Cross Validation Scores:  [0.84615385 0.96153846 0.96153846 0.92307692 0.88461538 0.88
 0.875      0.95833333 0.91666667 0.95833333]
Cross Validation mean:  0.9123589743589744


c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

## Ridge Regression

In [23]:
# Declare and fit the model
# Set regularization parameter
# turn down tolerance for short training time
clf_l2_LR = LogisticRegression(C=0.05, penalty='l2')

clf_l2_LR.fit(X_train, y_train)

print("Coefficient: ", clf_l2_LR.coef_)
print("Intercept:",clf_l2_LR.intercept_)
print("Train score with L2 penalty: %.4f" % clf_l2_LR.score(X_train, y_train))
print("Test score with L2 penalty: %.4f" % clf_l2_LR.score(X_test, y_test))
print("Cross Validation Scores: ",cross_val_score(clf_l2_LR, X_train, y_train, cv=10))
print("Cross Validation mean: ",cross_val_score(clf_l2_LR, X_train, y_train, cv=10).mean())

c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

Coefficient:  [[-0.00013725  0.01405026 -0.00696623 -0.03252093  0.07862523 -0.0250878
   0.01249791  0.03403386  0.06995623 -0.13536248]]
Intercept: [-1.21931884]
Train score with L2 penalty: 0.9044
Test score with L2 penalty: 0.8704
Cross Validation Scores:  [0.84615385 0.96153846 0.92307692 0.69230769 0.84615385 0.8
 0.83333333 0.91666667 0.875      0.91666667]
Cross Validation mean:  0.8610897435897436


c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

## Vanilla Regression

In [24]:
# Declare and fit the model
# Set regularization parameter
# turn down tolerance for short training time
clf_LR = LogisticRegression(C=0.05)

clf_LR.fit(X_train, y_train)


print("Coefficient: ", clf_LR.coef_)
print("Intercept:",clf_LR.intercept_)
print("Train score with no penalty: %.4f" % clf_LR.score(X_train, y_train))
print("Test score with no penalty: %.4f" % clf_LR.score(X_test, y_test))
print("Cross Validation Scores: ",cross_val_score(clf_LR, X_train, y_train, cv=10))
print("Cross Validation mean: ",cross_val_score(clf_LR, X_train, y_train, cv=10).mean())

c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Coefficient:  [[-0.00013725  0.01405026 -0.00696623 -0.03252093  0.07862523 -0.0250878
   0.01249791  0.03403386  0.06995623 -0.13536248]]
Intercept: [-1.21931884]
Train score with no penalty: 0.9044
Test score with no penalty: 0.8704


c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

Cross Validation Scores:  [0.84615385 0.96153846 0.92307692 0.69230769 0.84615385 0.8
 0.83333333 0.91666667 0.875      0.91666667]
Cross Validation mean:  0.8610897435897436


c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharath\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in

## Conclusion:

I would definitely choose the Lasso regresssion method as the score is realistic, even in terms of cross validation the range is acceptable.